<a href="https://colab.research.google.com/github/sdhar2020/Wrangling-Data/blob/master/Optimizing_Dataframes_and_Processing_in_Chunks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Processing large datasets can be challenge and require optimizing a dataframe's memory usage. Here data from [Lending Club](https://www.lendingclub.com), a marketplace for personal loans that matches borrowers with investors is used to explore the options.

Lending Club’s  massive data set is analyzed by optimizing the way that data is processed to reduce the load on memory, including optimizing the dataframe’s memory footprint and processing the data in chunks.

This is data is a csv file with 115 columns and 40.38 MB in size. This data is downloaded from https://data.world/jaypeedevlin/lending-club-loan-data-2007-11

In [70]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 99
datalink= 'https://query.data.world/s/vr4eky46ig6wa6ofatg3ucmw67422p'

first_five = pd.read_csv(datalink, nrows=5,skiprows=1)
first_five.shape

(5, 115)

In [71]:
sum(first_five.dtypes== 'object')

24

In [72]:
thousand_chunk = pd.read_csv(datalink, nrows=1000, skiprows=1)
thousand_chunk.memory_usage(deep=True).sum()/(1024*1024)

2.370248794555664

### Let's try tripling to 3000 rows and calculate the memory footprint for each chunk.

In [73]:
chunk_iter = pd.read_csv(datalink, chunksize=3000, skiprows=1)
for chunk in chunk_iter:
    print(chunk.memory_usage(deep=True).sum()/(1024*1024))

7.086687088012695
7.083115577697754
7.247514724731445
7.249743461608887
7.211755752563477
7.134672164916992
7.478672981262207
7.422393798828125
7.35462760925293
7.566001892089844
7.657600402832031
7.783655166625977
7.756294250488281
7.939258575439453
1.4041814804077148


## How many rows in the data set?

In [74]:
chunk_iter = pd.read_csv(datalink, chunksize=3000, skiprows=1)
total_rows = 0
for chunk in chunk_iter:
    total_rows += len(chunk)
print(total_rows)

42538


# Exploring the Data in Chunks

## How many columns have a numeric type? How many have a string type?

In [75]:
# Numeric columns
loans_chunks = pd.read_csv(datalink,chunksize=3000, skiprows=1)

numeric = []
string = []
for lc in loans_chunks:
    nums = lc.select_dtypes(include=[np.number]).shape[1]
    numeric.append(nums)
    strs = lc.select_dtypes(include=['object']).shape[1]
    string.append(strs)

print(numeric)
print(string)

[91, 91, 91, 91, 91, 92, 91, 92, 92, 92, 92, 92, 92, 90, 90]
[24, 24, 24, 24, 24, 23, 24, 23, 23, 23, 23, 23, 23, 25, 25]


In [76]:
# Are string columns consistent across chunks?
obj_cols = []
chunk_iter = pd.read_csv(datalink, chunksize=3000, skiprows=1)

for chunk in chunk_iter:
    chunk_obj_cols = chunk.select_dtypes(include=['object']).columns.tolist()
    # print(len(chunk.columns))
    if len(obj_cols) > 0:
        is_same = obj_cols == chunk_obj_cols
        if not is_same:
            print("overall obj cols Gap:", set(chunk_obj_cols).difference(set(obj_cols)), "\n")
            # print("chunk obj cols:", chunk_obj_cols, "\n")    
    else:
        obj_cols = chunk_obj_cols

overall obj cols Gap: set() 

overall obj cols Gap: set() 

overall obj cols Gap: set() 

overall obj cols Gap: set() 

overall obj cols Gap: set() 

overall obj cols Gap: set() 

overall obj cols Gap: set() 

overall obj cols Gap: {'id'} 

overall obj cols Gap: {'id'} 



##Observations:
 - By default -- 91 numeric columns and 24 string columns.
 - It seems like one column in particular (the `id` column) is being cast to int64 in the last 2 chunks but not in the earlier chunks. Since the `id` column won't be useful for analysis, visualization, or predictive modelling let's ignore this column.

## Unique values are there in each string column 
## String columns with 50% or less unique values

In [77]:

loans_chunks = pd.read_csv(datalink,chunksize=3000, skiprows=1)

uniques = {}
for lc in loans_chunks:
    strings_only = lc.select_dtypes(include=['object'])
    cols = strings_only.columns
    for c in cols:
        val_counts = strings_only[c].value_counts()
        if c in uniques:
            uniques[c].append(val_counts)
        else:
            uniques[c] = [val_counts]

uniques_combined = {}
unique_stats = {
    'column_name': [],
    'total_values': [],
    'unique_values': [],
}
for col in uniques:
    u_concat = pd.concat(uniques[col])
    u_group = u_concat.groupby(u_concat.index).sum()
    uniques_combined[col] = u_group
    if u_group.shape[0] < 50:
        print(col, u_group.shape[0])

term 2
grade 7
sub_grade 35
emp_length 11
home_ownership 5
verification_status 3
loan_status 9
pymnt_plan 2
purpose 14
initial_list_status 1
application_type 1


## Float columns have no missing values 
### Candidates for conversion to the integer type

In [78]:
loans_chunks = pd.read_csv(datalink,chunksize=3000, skiprows=1)
# missing dataframe for missing values
missing = []
for lc in loans_chunks:
    floats = lc.select_dtypes(include=['float'])
    # Append missing value count series, columns as index
    missing.append(floats.apply(pd.isnull).sum())
# Concat all missing value chunk series
combined_missing = pd.concat(missing)
# group by and sort by index
combined_missing.groupby(combined_missing.index).sum().sort_values()

loan_amnt                        3
installment                      3
last_pymnt_amnt                  3
member_id                        3
revol_bal                        3
                             ...  
mths_since_recent_inq        42538
mths_since_recent_bc_dlq     42538
mths_since_recent_bc         42538
num_actv_rev_tl              42538
verification_status_joint    42538
Length: 91, dtype: int64

## Calculate the total memory usage across all of the chunks.

In [79]:
loans_chunks = pd.read_csv(datalink,chunksize=3000, skiprows=1)

mem_usage = []

for lc in loans_chunks:
    mem_usage.append(lc.memory_usage(deep=True).sum() / 1024 ** 2)

sum(mem_usage)

105.37617492675781

## Optimizing String Columns

Determine which string columns can be converted to a numeric type once cleaned. Focusing on columns that would actually be useful for analysis and modelling.

In [80]:
obj_cols

['term',
 'int_rate',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'earliest_cr_line',
 'revol_util',
 'initial_list_status',
 'last_pymnt_d',
 'next_pymnt_d',
 'last_credit_pull_d',
 'application_type']

In [81]:
useful_obj_cols = ['term', 'sub_grade', 'emp_title', 'home_ownership', 'verification_status', 'issue_d', 'purpose', 'earliest_cr_line', 'revol_util', 'last_pymnt_d', 'last_credit_pull_d']

In [82]:
## Create dictionary (key: column, value: list of Series objects representing each chunk's value counts)
chunk_iter = pd.read_csv(datalink, chunksize=3000, skiprows=1)
str_cols_vc = {}
for chunk in chunk_iter:
    str_cols = chunk.select_dtypes(include=['object'])
    for col in str_cols.columns:
        current_col_vc = str_cols[col].value_counts()
        if col in str_cols_vc:
            str_cols_vc[col].append(current_col_vc)
        else:
            str_cols_vc[col] = [current_col_vc]

In [83]:
## Combine the value counts.
combined_vcs = {}

for col in str_cols_vc:
    combined_vc = pd.concat(str_cols_vc[col])
    final_vc = combined_vc.groupby(combined_vc.index).sum()
    combined_vcs[col] = final_vc

In [84]:
for col in useful_obj_cols:
    print(col)
    print(combined_vcs[col])
    print("-----------")

term
 36 months    31534
 60 months    11001
Name: term, dtype: int64
-----------
sub_grade
A1    1142
A2    1520
A3    1823
A4    2905
A5    2793
B1    1882
B2    2113
B3    2997
B4    2590
B5    2807
C1    2264
C2    2157
C3    1658
C4    1370
C5    1291
D1    1053
D2    1485
D3    1322
D4    1140
D5    1016
E1     884
E2     791
E3     668
E4     552
E5     499
F1     392
F2     308
F3     236
F4     211
F5     154
G1     141
G2     107
G3      79
G4      99
G5      86
Name: sub_grade, dtype: int64
-----------
emp_title
  old palm inc                       1
 Brocade Communications              1
 CenturyLink                         1
 Department of Homeland Security     1
 Down To Earth Distributors, Inc.    1
                                    ..
zashko inc.                          1
zeno office solutions                1
zion lutheran school                 1
zoll medical corp                    1
zozaya officiating                   1
Name: emp_title, Length: 30658, dtype: int

### Convert to category

In [85]:
convert_col_dtypes = {
    "sub_grade": "category", "home_ownership": "category", 
    "verification_status": "category", "purpose": "category"
}

- Convert `term` and `revol_util` to numerical by data cleaning.
- Convert `issue_d`, `earliest_cr_line`, `last_pymnt_d`, and `last_credit_pull_d` to datetime.

In [86]:
chunk[useful_obj_cols]

,term,sub_grade,emp_title,home_ownership,verification_status,issue_d,purpose,earliest_cr_line,revol_util,last_pymnt_d,last_credit_pull_d
42000,36 months,C2,Best Buy,RENT,Not Verified,Feb-2008,debt_consolidation,Jul-2000,100.7%,Feb-2011,Sep-2016
42001,36 months,G2,CVS PHARMACY,OWN,Not Verified,Feb-2008,debt_consolidation,Mar-1989,51.9%,Nov-2008,Sep-2016
42002,36 months,E4,General Motors,RENT,Not Verified,Feb-2008,debt_consolidation,Dec-1998,80.7%,Feb-2011,Sep-2016
42003,36 months,G4,usa medical center,RENT,Not Verified,Feb-2008,debt_consolidation,Jul-1995,57.2%,Feb-2011,Jun-2011
42004,36 months,B3,InvestSource Inc,RENT,Not Verified,Feb-2008,debt_consolidation,Sep-2005,74%,Mar-2010,Sep-2016
...,...,...,...,...,...,...,...,...,...,...,...
42533,36 months,B3,NaN,RENT,Not Verified,Jun-2007,other,NaN,NaN,Jun-2010,May-2007
42534,36 months,A5,NaN,NONE,Not Verified,Jun-2007,other,NaN,NaN,Jun-2010,Aug-2007
42535,36 months,A3,Homemaker,MORTGAGE,Not Verified,Jun-2007,other,NaN,NaN,Jun-2010,Feb-2015
42536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
chunk_iter = pd.read_csv(datalink, chunksize=3000, \
                         dtype=convert_col_dtypes, parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"], skiprows=1)

for chunk in chunk_iter:
    term_cleaned = chunk['term'].str.lstrip(" ").str.rstrip(" months")
    revol_cleaned = chunk['revol_util'].str.rstrip("%")
    chunk['term'] = pd.to_numeric(term_cleaned)
    chunk['revol_util'] = pd.to_numeric(revol_cleaned)
    
chunk.dtypes

id                             object
member_id                     float64
loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
                               ...   
tax_liens                     float64
tot_hi_cred_lim               float64
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
Length: 115, dtype: object

In [88]:
chunk_iter = pd.read_csv(datalink, chunksize=3000, dtype=convert_col_dtypes,\
                         parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"], skiprows=1)
mv_counts = {}
for chunk in chunk_iter:
    term_cleaned = chunk['term'].str.lstrip(" ").str.rstrip(" months")
    revol_cleaned = chunk['revol_util'].str.rstrip("%")
    chunk['term'] = pd.to_numeric(term_cleaned)
    chunk['revol_util'] = pd.to_numeric(revol_cleaned)
    float_cols = chunk.select_dtypes(include=['float'])
    for col in float_cols.columns:
        missing_values = len(chunk) - chunk[col].count()
        if col in mv_counts:
            mv_counts[col] = mv_counts[col] + missing_values
        else:
            mv_counts[col] = missing_values
mv_counts

{'acc_now_delinq': 32,
 'acc_open_past_24mths': 42538,
 'all_util': 42538,
 'annual_inc': 7,
 'annual_inc_joint': 42538,
 'avg_cur_bal': 42538,
 'bc_open_to_buy': 42538,
 'bc_util': 42538,
 'chargeoff_within_12_mths': 148,
 'collection_recovery_fee': 3,
 'collections_12_mths_ex_med': 148,
 'delinq_2yrs': 32,
 'delinq_amnt': 32,
 'dti': 3,
 'dti_joint': 42538,
 'fico_range_high': 3,
 'fico_range_low': 3,
 'funded_amnt': 3,
 'funded_amnt_inv': 3,
 'il_util': 42538,
 'inq_fi': 42538,
 'inq_last_12m': 42538,
 'inq_last_6mths': 32,
 'installment': 3,
 'last_fico_range_high': 3,
 'last_fico_range_low': 3,
 'last_pymnt_amnt': 3,
 'loan_amnt': 3,
 'max_bal_bc': 42538,
 'member_id': 3,
 'mo_sin_old_il_acct': 42538,
 'mo_sin_old_rev_tl_op': 42538,
 'mo_sin_rcnt_rev_tl_op': 42538,
 'mo_sin_rcnt_tl': 42538,
 'mort_acc': 42538,
 'mths_since_last_delinq': 26929,
 'mths_since_last_major_derog': 42538,
 'mths_since_last_record': 38887,
 'mths_since_rcnt_il': 42538,
 'mths_since_recent_bc': 42538,
 'mt

In [89]:
chunk_iter = pd.read_csv(datalink, chunksize=3000, dtype=convert_col_dtypes, \
                         parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"], skiprows=1)
mv_counts = {}
for chunk in chunk_iter:
    term_cleaned = chunk['term'].str.lstrip(" ").str.rstrip(" months")
    revol_cleaned = chunk['revol_util'].str.rstrip("%")
    chunk['term'] = pd.to_numeric(term_cleaned)
    chunk['revol_util'] = pd.to_numeric(revol_cleaned)
    chunk = chunk.dropna(how='all')
    float_cols = chunk.select_dtypes(include=['float'])
    for col in float_cols.columns:
        missing_values = len(chunk) - chunk[col].count()
        if col in mv_counts:
            mv_counts[col] = mv_counts[col] + missing_values
        else:
            mv_counts[col] = missing_values
mv_counts

{'acc_now_delinq': 32,
 'acc_open_past_24mths': 42538,
 'all_util': 42538,
 'annual_inc': 7,
 'annual_inc_joint': 42538,
 'avg_cur_bal': 42538,
 'bc_open_to_buy': 42538,
 'bc_util': 42538,
 'chargeoff_within_12_mths': 148,
 'collection_recovery_fee': 3,
 'collections_12_mths_ex_med': 148,
 'delinq_2yrs': 32,
 'delinq_amnt': 32,
 'dti': 3,
 'dti_joint': 42538,
 'fico_range_high': 3,
 'fico_range_low': 3,
 'funded_amnt': 3,
 'funded_amnt_inv': 3,
 'il_util': 42538,
 'inq_fi': 42538,
 'inq_last_12m': 42538,
 'inq_last_6mths': 32,
 'installment': 3,
 'last_fico_range_high': 3,
 'last_fico_range_low': 3,
 'last_pymnt_amnt': 3,
 'loan_amnt': 3,
 'max_bal_bc': 42538,
 'member_id': 3,
 'mo_sin_old_il_acct': 42538,
 'mo_sin_old_rev_tl_op': 42538,
 'mo_sin_rcnt_rev_tl_op': 42538,
 'mo_sin_rcnt_tl': 42538,
 'mort_acc': 42538,
 'mths_since_last_delinq': 26929,
 'mths_since_last_major_derog': 42538,
 'mths_since_last_record': 38887,
 'mths_since_rcnt_il': 42538,
 'mths_since_recent_bc': 42538,
 'mt